<a href="https://colab.research.google.com/github/TheRadDani/Wrapper-high-level-operations-considerations/blob/main/Serialize_Deserialize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional, LSTM

@tf.keras.utils.register_keras_serializable()
class Custom_Layer(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Custom_Layer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='glorot_uniform',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w)

In [10]:
import os

model_path = 'my_model.h5'

if os.path.exists(model_path):
    os.remove(model_path)

inputs = tf.keras.Input(shape=(10,))
x = tf.keras.layers.Reshape((1, 10))(inputs) # reshape to (samples, 1, features)
x = Bidirectional(LSTM(32))(x)
x = Custom_Layer(16)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.save(model_path)
loaded_model = tf.keras.models.load_model(model_path)
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
loaded_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 10)]              0         
                                                                 
 reshape_4 (Reshape)         (None, 1, 10)             0         
                                                                 
 bidirectional_5 (Bidirectio  (None, 64)               11008     
 nal)                                                            
                                                                 
 custom__layer_2 (Custom_Lay  (None, 16)               1024      
 er)                                                             
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 12,049
Trainable params: 12,049
Non-trainable